# Chain restaurant opportunities at greater Helsinki mass transit hubs
### Applied Data Science Capstone for IBM/Coursera by Pauli Hannola

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

In this project we are performing exploratory analysis of restaurants and cafes near the four biggest mass transit hubs in the greater Helsinki area, namely **Helsinki Central Railway station**, **Helsinki Bus Station**, **Pasila Railway Station** and **Helsinki Airport**. The stakeholders are interested in finding opportunities for a small chain of restaurants or cafes at these places of high passenger volumes and identifying what kind of services are lacking in these areas.

## Data <a name="data"></a>

Let's first import the necessary libraries and one helper function for our notebook.

In [125]:
from geopy.geocoders import Nominatim 
import folium 
import requests
import pandas as pd
from pandas.io.json import json_normalize

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Let's now use Nominatim to get the coordinates of Helsinki Central station.

In [126]:
address = 'Helsinki central station'

geolocator = Nominatim(user_agent="to_explorer")
hcs_location = geolocator.geocode(address)
hcs_latitude = hcs_location.latitude
hcs_longitude = hcs_location.longitude
print('The geograpical coordinates of Helsinki Central station are {}, {}.'.format(hcs_latitude, hcs_longitude))

The geograpical coordinates of Helsinki Central station are 60.1713198, 24.9414566.


We are going to repeat the process to get the locations of our three other hubs. To get the coordinates for the Helsinki Bus station we had to use the search keyword "Kamppi", since the station is located in the Kamppi shopping centre.

In [127]:
address = 'Kamppi'

geolocator = Nominatim(user_agent="to_explorer")
hbs_location = geolocator.geocode(address)
hbs_latitude = hbs_location.latitude
hbs_longitude = hbs_location.longitude
print('The geograpical coordinates of Helsinki bus station are {}, {}.'.format(hbs_latitude, hbs_longitude))

address = 'Pasila railway station'

geolocator = Nominatim(user_agent="to_explorer")
prs_location = geolocator.geocode(address)
prs_latitude = prs_location.latitude
prs_longitude = prs_location.longitude
print('The geograpical coordinates of Pasila railway station are {}, {}.'.format(prs_latitude, prs_longitude))

address = 'Helsinki airport'

geolocator = Nominatim(user_agent="to_explorer")
ha_location = geolocator.geocode(address)
ha_latitude = ha_location.latitude
ha_longitude = ha_location.longitude
print('The geograpical coordinates of Helsinki airport are {}, {}.'.format(ha_latitude, ha_longitude))

The geograpical coordinates of Helsinki bus station are 60.1685348, 24.9304942.
The geograpical coordinates of Pasila railway station are 60.197873, 24.9322653.
The geograpical coordinates of Helsinki airport are 60.32135905, 24.9472068000384.


We're gonna put our Foursquare credentials in and then do an explore call to the Foursquare API about the first hub and store the results in a pandas dataframe.

In [128]:
CLIENT_ID = 'F12LR0WGUSFUV0HFIIB13O15XVOSKCR1VRJVW4TR0MZ33EYN' # your Foursquare ID
CLIENT_SECRET = 'DP1DDMO2R3TFN5EXBESLT32IFQDGJZHDL2HY2JCQBMNSULSK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, hcs_latitude, hcs_longitude, VERSION, radius, LIMIT)

results = requests.get(url).json()
items = results['response']['groups'][0]['items']

dataframe1 = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe1.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered1 = dataframe1.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered1['venue.categories'] = dataframe_filtered1.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered1.columns = [col.split('.')[-1] for col in dataframe_filtered1.columns]

dataframe_filtered1.head()



,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Habitat Helsinki,Furniture / Home Store,Kaivokatu 8,FI,Helsinki,Suomi,NaN,203,"[Kaivokatu 8, 00100 Helsinki, Suomi]","[{'label': 'display', 'lat': 60.16955676588021...",60.169557,24.942441,NaN,00100,Uusimaa,54702b87498e96a8900e105b
1,Suomen Kansallisteatteri,Theater,Läntinen Teatterikuja 1,FI,Helsinki,Suomi,Rautatientori,183,"[Läntinen Teatterikuja 1 (Rautatientori), 0010...","[{'label': 'display', 'lat': 60.17251, 'lng': ...",60.172510,24.943740,NaN,00100,Uusimaa,4adcdb22f964a520666021e3
2,Ateneum,Art Museum,Kaivokatu 2,FI,Helsinki,Suomi,NaN,171,"[Kaivokatu 2, 00100 Helsinki, Suomi]","[{'label': 'display', 'lat': 60.17030164536989...",60.170302,24.943788,NaN,00100,Uusimaa,4adcdb23f964a520af6021e3
3,Fantasiapelit,Hobby Shop,Kaivopiha,FI,Helsinki,Suomi,NaN,213,"[Kaivopiha, 00100 Helsinki, Suomi]","[{'label': 'display', 'lat': 60.16946082452627...",60.169461,24.940527,NaN,00100,Uusimaa,4bd59544cfa7b71327fc25da
4,Pien Shop & Bar,Beer Store,Ateneuminkuja 2,FI,Helsinki,Suomi,NaN,249,"[Ateneuminkuja 2, 00100 Helsinki, Suomi]","[{'label': 'display', 'lat': 60.16967317676566...",60.169673,24.944526,NaN,00100,Uusimaa,57ab4c95498ef8003be05e61


We have a list now of 100 venues close to the location. A lot of the information stored, however, is not pertinent to our investigation, so we're going to copy only the name and categories into a new dataframe.

In [129]:
df1 = dataframe_filtered1[['name','categories']]
df1.head()

,name,categories
0,Habitat Helsinki,Furniture / Home Store
1,Suomen Kansallisteatteri,Theater
2,Ateneum,Art Museum
3,Fantasiapelit,Hobby Shop
4,Pien Shop & Bar,Beer Store


Next, we are gonna count the number of each type of venue.

In [130]:
df1 = df1['categories'].value_counts().to_frame()
df1

,categories
Café,7
Clothing Store,6
Coffee Shop,5
Bakery,3
Wine Bar,3
Furniture / Home Store,3
Art Museum,3
Burger Joint,3
Hawaiian Restaurant,2
Restaurant,2


df1 is now our dataframe containing the number of each type of venue near Helsinki Central Station. We will now repeat the process for the three other transit hubs and store their info into dataframes named df2, df3 and df4.

In [131]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, hbs_latitude, hbs_longitude, VERSION, radius, LIMIT)
results = requests.get(url).json()
items = results['response']['groups'][0]['items']
dataframe1 = json_normalize(items)
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe1.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered1 = dataframe1.loc[:, filtered_columns]
dataframe_filtered1['venue.categories'] = dataframe_filtered1.apply(get_category_type, axis=1)
dataframe_filtered1.columns = [col.split('.')[-1] for col in dataframe_filtered1.columns]
df2 = dataframe_filtered1[['name','categories']]
df2 = df2['categories'].value_counts().to_frame()
df2.head()

,categories
Scandinavian Restaurant,5
Café,4
Chinese Restaurant,3
Art Museum,3
Wine Bar,3


In [132]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, prs_latitude, prs_longitude, VERSION, radius, LIMIT)
results = requests.get(url).json()
items = results['response']['groups'][0]['items']
dataframe1 = json_normalize(items)
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe1.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered1 = dataframe1.loc[:, filtered_columns]
dataframe_filtered1['venue.categories'] = dataframe_filtered1.apply(get_category_type, axis=1)
dataframe_filtered1.columns = [col.split('.')[-1] for col in dataframe_filtered1.columns]
df3 = dataframe_filtered1[['name','categories']]
df3 = df3['categories'].value_counts().to_frame()
df3.head()

,categories
Sandwich Place,1
Health Food Store,1
Coffee Shop,1
Thai Restaurant,1
Massage Studio,1


In [133]:
radius = 2000 # For the airport we expand the search radius because of the scale of the place

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, ha_latitude, ha_longitude, VERSION, radius, LIMIT)
results = requests.get(url).json()
items = results['response']['groups'][0]['items']
dataframe1 = json_normalize(items)
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe1.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered1 = dataframe1.loc[:, filtered_columns]
dataframe_filtered1['venue.categories'] = dataframe_filtered1.apply(get_category_type, axis=1)
dataframe_filtered1.columns = [col.split('.')[-1] for col in dataframe_filtered1.columns]
df4 = dataframe_filtered1[['name','categories']]
df4 = df4['categories'].value_counts().to_frame()
df4.head()

,categories
Airport Service,12
Airport Lounge,5
Coffee Shop,4
Scandinavian Restaurant,3
Hotel,2


We add out dataframes df1 to df4 together into a new dataframe df5.

In [134]:
df5 = df1.add(df2, fill_value=0)
df5 = df5.add(df3, fill_value=0)
df5 = df5.add(df4, fill_value=0)
df5.shape

(115, 1)

Our df5-dataframe contains rows that are not restaurants or cafes. We're going to drop those rows next. Unfortunately we have to do this pretty much by hand.

In [135]:
df5 = df5.drop("Airport", axis=0)
df5 = df5.drop("Airport Lounge", axis=0)
df5 = df5.drop("Airport Service", axis=0)
df5 = df5.drop("Art Gallery", axis=0)
df5 = df5.drop("Art Museum", axis=0)
df5 = df5.drop("Arts & Crafts Store", axis=0)
df5 = df5.drop("Bed & Breakfast", axis=0)
df5 = df5.drop("Bookstore", axis=0)
df5 = df5.drop("Boutique", axis=0)
df5 = df5.drop("Circus", axis=0)
df5 = df5.drop("Climbing Gym", axis=0)
df5 = df5.drop("Clothing Store", axis=0)
df5 = df5.drop("Concert Hall", axis=0)
df5 = df5.drop("Convenience Store", axis=0)
df5 = df5.drop("Dance Studio", axis=0)
df5 = df5.drop("Department Store", axis=0)
df5 = df5.drop("Duty-free Shop", axis=0)
df5 = df5.drop("Electronics Store", axis=0)
df5 = df5.drop("Road", axis=0)
df5 = df5.drop("Rock Club", axis=0)
df5 = df5.drop("Salon / Barbershop", axis=0)
df5 = df5.drop("Scenic Lookout", axis=0)
df5 = df5.drop("Science Museum", axis=0)
df5 = df5.drop("Shoe Store", axis=0)
df5 = df5.drop("Sporting Goods Shop", axis=0)
df5 = df5.drop("Supermarket", axis=0)
df5 = df5.drop("Theater", axis=0)
df5 = df5.drop("Toy / Game Store", axis=0)
df5 = df5.drop("Train Station", axis=0)
df5 = df5.drop("Wine Shop", axis=0)
df5 = df5.drop("Yoga Studio", axis=0)
df5 = df5.drop("Flower Shop", axis=0)
df5 = df5.drop("Furniture / Home Store", axis=0)
df5 = df5.drop("Garden", axis=0)
df5 = df5.drop("Nightclub", axis=0)
df5 = df5.drop("Organic Grocery", axis=0)
df5 = df5.drop("Outdoor Supply Store", axis=0)
df5 = df5.drop("Park", axis=0)
df5 = df5.drop("Performing Arts Venue", axis=0)
df5 = df5.drop("Playground", axis=0)
df5 = df5.drop("Plaza", axis=0)
df5 = df5.drop("Pool", axis=0)
df5 = df5.drop("Pool Hall", axis=0)
df5 = df5.drop("Racetrack", axis=0)
df5 = df5.drop("Record Shop", axis=0)
df5 = df5.drop("Gift Shop", axis=0)
df5 = df5.drop("Grocery Store", axis=0)
df5 = df5.drop("Gym", axis=0)
df5 = df5.drop("Gym / Fitness Center", axis=0)
df5 = df5.drop("Health Food Store", axis=0)
df5 = df5.drop("Hobby Shop", axis=0)
df5 = df5.drop("Hotel", axis=0)
df5 = df5.drop("Hotel Bar", axis=0)
df5 = df5.drop("IT Services", axis=0)
df5 = df5.drop("Indie Movie Theater", axis=0)
df5 = df5.drop("Kids Store", axis=0)
df5 = df5.drop("Lingerie Store", axis=0)
df5 = df5.drop("Massage Studio", axis=0)
df5 = df5.drop("Men's Store", axis=0)
df5 = df5.drop("Miscellaneous Shop", axis=0)
df5 = df5.drop("Movie Theater", axis=0)
df5 = df5.drop("Multiplex", axis=0)
df5 = df5.drop("Bus Stop", axis=0)
df5 = df5.drop("Comedy Club", axis=0)
df5 = df5.drop("Shoe Repair", axis=0)
print(df5.to_string())

                               categories
Asian Restaurant                      2.0
Bakery                                5.0
Bar                                   1.0
Beer Bar                              3.0
Beer Store                            1.0
Bistro                                1.0
Brewery                               3.0
Burger Joint                          4.0
Burrito Place                         1.0
Café                                 12.0
Chinese Restaurant                    3.0
Cocktail Bar                          3.0
Coffee Shop                          12.0
Deli / Bodega                         1.0
Dumpling Restaurant                   1.0
Falafel Restaurant                    2.0
Fast Food Restaurant                  2.0
Filipino Restaurant                   2.0
Fish & Chips Shop                     1.0
Food Court                            2.0
French Restaurant                     1.0
Gourmet Shop                          1.0
Hawaiian Restaurant               

After a moment or two of furious copy-pasting, we now have a list of only the cafes and restaurants near our four transit hubs. Since that list only represents types of venues that exist, it is not fully representative of the entire range of business opportunities. For that, we need to also get information on the types of venues that DON'T exist near the hubs yet. Next, we are going to use the Foursquare API again to get a list of all possible restaurant and cafe venue types. Due to the structure of the Foursquare venue list, we will do one API call and then wrangle data from three different spots of the response into separate dataframes df_cat1, df_cat2 and df_cat3.

In [136]:
url = 'https://api.foursquare.com/v2/venues/categories?client_id={}&client_secret={}&v={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION)
results = requests.get(url).json()
items = results['response']['categories'][3]['categories']
dataframe = json_normalize(items)
df_cat1 = dataframe['name']

items = results['response']['categories'][4]['categories'][0]['categories']
dataframe = json_normalize(items)
df_cat2 = dataframe['name']

items = results['response']['categories'][4]['categories']
dataframe = json_normalize(items)
df_cat3 = dataframe['name']

Then we use the pandas concatenate function to combine the three lists into a final one, called df_cat4.

In [137]:
frames = [df_cat1, df_cat2, df_cat3]

df_cat4 = pd.concat(frames, ignore_index=True, sort=False)
df_cat4

0                  Afghan Restaurant
1                 African Restaurant
2                American Restaurant
3                   Asian Restaurant
4              Australian Restaurant
5                Austrian Restaurant
6                          BBQ Joint
7                         Bagel Shop
8                             Bakery
9                 Belgian Restaurant
10                            Bistro
11                    Breakfast Spot
12                   Bubble Tea Shop
13                            Buffet
14                      Burger Joint
15                         Cafeteria
16                              Café
17         Cajun / Creole Restaurant
18              Caribbean Restaurant
19              Caucasian Restaurant
20                       Coffee Shop
21           Comfort Food Restaurant
22                          Creperie
23                  Czech Restaurant
24                     Deli / Bodega
25                      Dessert Shop
26                             Diner
2

We turn df_cat4, which is a series to a pandas dataframe called df_cat5. From this dataframe now, we are going to delete all the entries that appear in our earlier, df5-dataframe, so we end up with a list of the types of places that do not exist near our transit hubs.

In [138]:
df_cat5 = df_cat4.to_frame()
for row in df_cat5.itertuples():
    if row.name in df5.index:
        df_cat5.drop([row.Index], inplace=True)
df_cat5.sort_values('name', inplace=True)
df_cat5.reset_index(drop=True, inplace=True)
df_cat5

,name
0,Afghan Restaurant
1,African Restaurant
2,American Restaurant
3,Australian Restaurant
4,Austrian Restaurant
5,BBQ Joint
6,Bagel Shop
7,Beach Bar
8,Beer Garden
9,Belgian Restaurant


Now that the df_cat5 -dataframe is ready (and alphabetized) for our final report, we turn back to the list of existing venues, which we called df5. We are going to remove all the rows, where the number of venues exceeds 2, therefore getting a list of the types of places with only a little competition in the transit hub areas.

In [139]:
for row in df5.itertuples():
    if row.categories > 2:
        df5.drop([row.Index], inplace=True)
df5

,categories
Asian Restaurant,2.0
Bar,1.0
Beer Store,1.0
Bistro,1.0
Burrito Place,1.0
Deli / Bodega,1.0
Dumpling Restaurant,1.0
Falafel Restaurant,2.0
Fast Food Restaurant,2.0
Filipino Restaurant,2.0


We will drop the column with the numbers now to get a nicer looking list of just the venue types.

In [140]:
df5.drop(['categories'], axis=1, inplace=True)
df5

""
Asian Restaurant
Bar
Beer Store
Bistro
Burrito Place
Deli / Bodega
Dumpling Restaurant
Falafel Restaurant
Fast Food Restaurant
Filipino Restaurant


We'll just set the name column as the index on df_cat5, to make it look similar to df5.

In [141]:
df_cat5.set_index('name', inplace=True)
df_cat5

""
name
Afghan Restaurant
African Restaurant
American Restaurant
Australian Restaurant
Austrian Restaurant
BBQ Joint
Bagel Shop
Beach Bar
Beer Garden


For our final result, we will print the two lists; df5 now containing a list of venue types with 1 or 2 venues combined near our mass transit hubs and df_cat5 containing a list of venue types with no venues near the hubs.

In [142]:
print("Venue types with little competition: \n")
names = df5.index.values
print(', '.join(names))
print("\nVenue types with no competition:  \n")
names = df_cat5.index.values
print(', '.join(names))

Venue types with little competition: 

Asian Restaurant, Bar, Beer Store, Bistro, Burrito Place, Deli / Bodega, Dumpling Restaurant, Falafel Restaurant, Fast Food Restaurant, Filipino Restaurant, Fish & Chips Shop, Food Court, French Restaurant, Gourmet Shop, Himalayan Restaurant, Hot Dog Joint, Indian Restaurant, Irish Pub, Malay Restaurant, Mexican Restaurant, Middle Eastern Restaurant, Modern European Restaurant, Moroccan Restaurant, Pastry Shop, Pet Café, Pub, Salad Place, Seafood Restaurant, Smoothie Shop, Sushi Restaurant, Tea Room, Vegetarian / Vegan Restaurant, Vietnamese Restaurant

Venue types with no competition:  

Afghan Restaurant, African Restaurant, American Restaurant, Australian Restaurant, Austrian Restaurant, BBQ Joint, Bagel Shop, Beach Bar, Beer Garden, Belgian Restaurant, Breakfast Spot, Bubble Tea Shop, Buffet, Cafeteria, Cajun / Creole Restaurant, Caribbean Restaurant, Caucasian Restaurant, Champagne Bar, Comfort Food Restaurant, Creperie, Czech Restaurant, Des